# `mammos_entity` quickstart

`mammos_entity` provides tools to work with ontology-enriched data. This is in particular important for data storage and exchange. Data exchange includes both exchanging files as well as passing data between (Python) functions.

In [1]:
from pathlib import Path

import mammos_entity as me
from mammos_entity import units as u  # for convenience mammos_entity re-exports mammos_units

## `Entity`

The central object is the `Entity` class, which combines quantities (values and unit) with their definition in an ontology (typically [EMMO](https://github.com/emmo-repo/EMMO) or the [MaMMoS additions for magnetic materials](https://github.com/MaMMoS-project/MagneticMaterialsOntology)).

Creating an entity can take the following arguments:
- the ontology label
- the value(s)
- the unit (optional, if not provided default units are taken from the ontology)
- a description (optional, empty string if not provided)

In [2]:
me.Entity("SpontaneousMagnetization", 0.7, "MA/m", "Measured with technique ABC")

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(0.7), unit='MA / m', description='Measured with technique ABC')

For entities commonly used in a magnetics context short-hand functions are provided:

In [3]:
me.Ms(0.7, "MA/m")

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(0.7), unit='MA / m')

Data can have arbitrary shape:

In [4]:
Ms = me.Ms([1, 2, 3], description="Measured with technique ABC")
Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2., 3.]), unit='A / m', description='Measured with technique ABC')

The entity provides a number of attributes to access ontology information:

In [5]:
Ms.ontology_label

'SpontaneousMagnetization'

In [6]:
Ms.ontology_label_with_iri

'SpontaneousMagnetization https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25'

In [7]:
print(Ms.ontology.elucidation[0])

The spontaneous magnetization, Ms, of a ferromagnet is the result
of alignment of the magnetic moments of individual atoms. Ms exists
within a domain of a ferromagnet.


Entities are immutable. To work with the underlying data, use the embedded `mammos_units.Quantity` object:

In [8]:
Ms.q

<Quantity [1., 2., 3.] A / m>

In [9]:
Ms.q * 2

<Quantity [2., 4., 6.] A / m>

If you only need the numerical values use:

In [10]:
Ms.value

array([1., 2., 3.])

## Searching for available entities

Several functions are provided to search for available entities in the ontology.

To search for all ontology labels (prefLabel and altLabels) that contain a given substring use:

In [11]:
me.search_labels("CurieTemperature")

['CurieTemperature']

In [12]:
me.search_labels("Magnetization")

['MagneticMomementPerUnitMass', 'Magnetization', 'SpontaneousMagnetization']

In the second example we see one surprising entry: "MagneticMomentPerUnitMass" does not contain the substring "Magnetization". It appears in the results because one of the altLabels contains the word "Magnetization":

In [13]:
me.mammos_ontology.MagneticMomementPerUnitMass.altLabel

['sigma, MassMagnetization, SpecificMagneticMoment']

## `EntityCollection`

Multiple entities can be grouped into an `EntityCollection`. The collection additionally carries a `description` (empty string by default).


In [14]:
me.EntityCollection(
    Ms=me.Ms([600, 500, 400], "kA/m"),
    T=me.T([100, 200, 300], "K"),
)

EntityCollection(
    description='',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 500., 400.]), unit='kA / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([100., 200., 300.]), unit='K'),
)

`EntityCollection` is not limited to `Entity` objects. It can also store `mammos_units.Quantity` objects or any other scalar or array-like object. We summarize these with the term "entity-like":

In [15]:
collection = me.EntityCollection(
    description="Some random test data",
    Ms=Ms,  # Entity
    T=me.T([100, 200, 300], "K"),  # Entity
    x=[1, 2, 3] * u.mm,  # Quantity
    comment=["", "", "defect in the sample"],  # list
)
collection

EntityCollection(
    description='Some random test data',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2., 3.]), unit='A / m', description='Measured with technique ABC'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([100., 200., 300.]), unit='K'),
    x=<Quantity [1., 2., 3.] mm>,
    comment=['', '', 'defect in the sample'],
)

Entity-likes in a collection do not have to have the same shape. 

Individual elements of the collection can be accessed as follows:

In [16]:
collection.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2., 3.]), unit='A / m', description='Measured with technique ABC')

In [17]:
collection.x

<Quantity [1., 2., 3.] mm>

If all entity-likes are one-dimensional and have the same length, the collection can be converted into a `pandas.DataFrame`:

In [18]:
collection.to_dataframe(include_units=True)

,Ms (A / m),T (K),x (mm),comment
0,1.0,100.0,1.0,
1,2.0,200.0,2.0,
2,3.0,300.0,3.0,defect in the sample


We can add a new element to the collection using:

In [19]:
collection.Tc = me.Tc(1000, "K")
collection

EntityCollection(
    description='Some random test data',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2., 3.]), unit='A / m', description='Measured with technique ABC'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([100., 200., 300.]), unit='K'),
    x=<Quantity [1., 2., 3.] mm>,
    comment=['', '', 'defect in the sample'],
    Tc=Entity(ontology_label='CurieTemperature', value=np.float64(1000.0), unit='K'),
)

## Reading and writing files

`mammos_entity` defines a yaml file format to store entities or an entity collection. The logical structure resembles the `EntityCollection`.

To write individual entities to file use `entities_to_file`. The first argument is the file name, all entities are passed as keyword arguments. Optionally, the file can also carry a description:

In [20]:
me.io.entities_to_file("file1.yaml", Ms=Ms, Tc=me.Tc([100, 100, 100]))

The file has the following content:

In [21]:
print(Path("file1.yaml").read_text())

metadata:
  version: v2
  description: ''
data:
  Ms:
    ontology_label: SpontaneousMagnetization
    description: Measured with technique ABC
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: A / m
    value: [1.0, 2.0, 3.0]
  Tc:
    ontology_label: CurieTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_6b5af5a8_a2d8_4353_a1d6_54c9f778343d
    unit: K
    value: [100.0, 100.0, 100.0]



The file can be read using `entities_from_file`. The returned object is always an `EntityCollection`:

In [22]:
me.io.entities_from_file("file1.yaml")

EntityCollection(
    description='',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2., 3.]), unit='A / m', description='Measured with technique ABC'),
    Tc=Entity(ontology_label='CurieTemperature', value=array([100., 100., 100.]), unit='K'),
)

`EntityCollection`s provide a method to write them to file. We only have to pass the filename:

In [23]:
collection.to_file("file2.yaml")

The file has the following content:

In [24]:
print(Path("file2.yaml").read_text())

metadata:
  version: v2
  description: Some random test data
data:
  Ms:
    ontology_label: SpontaneousMagnetization
    description: Measured with technique ABC
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: A / m
    value: [1.0, 2.0, 3.0]
  T:
    ontology_label: ThermodynamicTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [100.0, 200.0, 300.0]
  x:
    ontology_label: null
    description: ''
    ontology_iri: null
    unit: mm
    value: [1.0, 2.0, 3.0]
  comment:
    ontology_label: null
    description: ''
    ontology_iri: null
    unit: null
    value: ['', '', defect in the sample]
  Tc:
    ontology_label: CurieTemperature
    description: ''
    ontology_iri: https://w3id.org/emmo#EMMO_6b5af5a8_a2d8_4353_a1d6_54c9f778343d
    unit: K
    value: 1000.0



The file can be read using `entities_from_file`. The returned object is always an `EntityCollection`:

In [25]:
me.io.entities_from_file("file2.yaml")

EntityCollection(
    description='Some random test data',
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2., 3.]), unit='A / m', description='Measured with technique ABC'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([100., 200., 300.]), unit='K'),
    x=<Quantity [1., 2., 3.] mm>,
    comment=['', '', 'defect in the sample'],
    Tc=Entity(ontology_label='CurieTemperature', value=np.float64(1000.0), unit='K'),
)